In [1]:
!pip install -q openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from openai import OpenAI
import os

# DeepInfra api key: https://deepinfra.com/microsoft/WizardLM-2-8x22B
def load_model():
    client = OpenAI(
        api_key="6WfiI",
        base_url="https://api.deepinfra.com/v1/openai",
    )
    return client

client = load_model()
model_name="microsoft/WizardLM-2-8x22B"

In [12]:
class PromptTemplateGenerator:
    @staticmethod
    def generate_zero_shot_summary_prompt(text):
        return f"""
        Summarize the following text in a clear and concise manner.

        Text:
        {text}
        """

    @staticmethod
    def generate_one_shot_summary_prompt(text):
        return f"""
        Summarize the text based on this example:

        Example:
        Original: "Artificial intelligence is rapidly transforming various industries, from healthcare to finance. Machine learning algorithms can now diagnose diseases, predict market trends, and automate complex tasks."
        Summary: AI is revolutionizing multiple sectors by enabling advanced diagnostic, predictive, and automation capabilities.

        Now summarize this text:
        {text}
        """

    @staticmethod
    def generate_few_shot_summary_prompt(text):
        return f"""
        Summarize the text based on these examples:

        Example 1:
        Original: "Artificial intelligence is rapidly transforming various industries, from healthcare to finance. Machine learning algorithms can now diagnose diseases, predict market trends, and automate complex tasks."
        Summary: AI is revolutionizing multiple sectors by enabling advanced diagnostic, predictive, and automation capabilities.

        Example 2:
        Original: "Climate change poses significant challenges to global ecosystems. Rising temperatures are causing polar ice caps to melt, sea levels to rise, and extreme weather events to become more frequent."
        Summary: Climate change is threatening global environments through rising temperatures, melting ice caps, sea level increases, and more frequent extreme weather.

        Now summarize this text:
        {text}
        """

    @staticmethod
    def generate_chain_of_thought_summary_prompt(text):
        return f"""
        Let's summarize this text step by step:

        Text:
        {text}

        Reasoning Process:
        1. Initial Comprehension:
        - What is the main topic of the text?
        - What are the key points?

        2. Identify Core Ideas:
        - List the most important concepts.
        - Determine the primary message.

        3. Synthesis:
        - Combine key points into a coherent overview.
        - Ensure the summary captures the essence of the original text.

        4. Conciseness Check:
        - Verify the summary is within 2-3 sentences.
        - Eliminate any redundant phrases.
        - Ensure key points are preserved.
        - Remove unnecessary adjectives or adverbs.

        Structured Summary:
        """

    @staticmethod
    def generate_role_based_summary_prompt(text):
        return f"""
        As a content summarization expert, generate a concise and accurate summary of the following text. Focus on the main topic, key points, and primary message. Avoid including minor details or subjective interpretations.

        Text:
        {text}

        Instructions:
        - Identify the central theme.
        - Highlight only the core ideas.
        - Present the summary in a single, clear paragraph.

        Your Summary:
        """

In [13]:
def process_text(text, prompt_type, temperature=0.1):
    if prompt_type == "zero_shot":
        message_content = PromptTemplateGenerator.generate_zero_shot_summary_prompt(text)
    elif prompt_type == "one_shot":
        message_content = PromptTemplateGenerator.generate_one_shot_summary_prompt(text)
    elif prompt_type == "few_shot":
        message_content = PromptTemplateGenerator.generate_few_shot_summary_prompt(text)
    elif prompt_type == "chain_of_thought":
        message_content = PromptTemplateGenerator.generate_chain_of_thought_summary_prompt(text)
    elif prompt_type == "role_based":
        message_content = PromptTemplateGenerator.generate_role_based_summary_prompt(text)
    else:
        raise ValueError(f"Unknown prompt type: {prompt_type}")
    
    response = client.chat.completions.create(
        model= model_name,
        messages=[{"role": "user", "content": message_content}],
        temperature=temperature,
        max_tokens=256,
    )
    generated_text = response.choices[0].message.content
    return generated_text

In [25]:
text = """
Quantum computing is an area of computing focused on developing computers based on the principles of quantum theory. 
This theory explains the behavior of energy and material on atomic and subatomic levels. Quantum computing uses 
quantum bits, or qubits, which allow for a state of superposition, enabling them to perform computations at 
incredible speeds compared to classical computers. Researchers believe that quantum computing could solve 
complex problems much faster than today's classical computers, particularly in fields like cryptography, 
material science, and artificial intelligence.
"""

In [20]:
result = process_text(text, "zero_shot")
print(result)

 Quantum computing harnesses quantum theory to process information at the atomic and subatomic levels using quantum bits (qubits). Unlike classical computers, qubits can exist in multiple states simultaneously (superposition), allowing for significantly faster computations. This advanced technology has the potential to revolutionize fields such as cryptography, material science, and artificial intelligence by solving complex problems more efficiently than current computers.


In [21]:
result = process_text(text, "one_shot")
print(result)

 Quantum computing harnesses quantum theory to process information at unprecedented speeds using qubits in superposition, offering potential breakthroughs in cryptography, material science, and AI by solving complex problems faster than classical computers.


In [22]:
result = process_text(text, "few_shot")
print(result)

 Quantum computing harnesses quantum theory to process information at unprecedented speeds using qubits in superposition, offering potential breakthroughs in solving complex problems across cryptography, material science, and AI, far exceeding classical computer capabilities.


In [23]:
result = process_text(text, "chain_of_thought")
print(result)

 Structured Summary:

Quantum computing represents a revolutionary approach to computation, leveraging the principles of quantum theory to process information at the atomic and subatomic levels. This field centers on the use of qubits, which can exist in multiple states simultaneously, vastly increasing computational efficiency. Quantum computers hold the promise of tackling intricate problems across various domains, including cryptography, material science, and AI, at speeds unattainable by classical computers. The primary message of the text is that quantum computing, by harnessing the unique properties of quantum mechanics, has the potential to transform the landscape of computation and solve problems that are currently intractable for classical systems.


In [24]:
result = process_text(text, "role_based")
print(result)

 Quantum computing represents a revolutionary approach to computation that leverages the principles of quantum theory to process information at the atomic and subatomic levels. Unlike classical computers, quantum computers utilize qubits, which can exist in multiple states simultaneously—a phenomenon known as superposition—enabling them to execute complex calculations at unprecedented speeds. This advanced computational capability holds the promise of significantly accelerating problem-solving in various domains, including cryptography, material science, and artificial intelligence, where classical computers face substantial limitations.
